# Markov Model Analysis Example

This notebook demonstrates how to perform Markov model analysis for health economic evaluation.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os

# Add the scripts directory to the path
sys.path.append(os.path.join(os.pardir, 'scripts'))
sys.path.append(os.path.join(os.pardir, 'scripts', 'models'))
sys.path.append(os.path.join(os.pardir, 'scripts', 'core'))

In [ ]:
# Define Markov model functions
def create_markov_transition_matrix(health_states, transition_probs):
    """
    Create a Markov transition matrix from transition probabilities.
    
    Args:
        health_states: List of health state names
        transition_probs: Dictionary of transition probabilities
        
    Returns:
        Transition probability matrix
    """
    n_states = len(health_states)
    tm = np.zeros((n_states, n_states))
    
    # Convert state names to indices
    state_idx = {state: i for i, state in enumerate(health_states)}
    
    # Populate the transition matrix
    for (from_state, to_state), prob in transition_probs.items():
        from_idx = state_idx[from_state]
        to_idx = state_idx[to_state]
        tm[from_idx, to_idx] = prob
    
    # Ensure each row sums to 1
    for i in range(n_states):
        row_sum = tm[i].sum()
        if row_sum != 0 and row_sum != 1.0:
            # Normalize if probabilities don't sum to 1
            tm[i] = tm[i] / row_sum
    
    return tm, state_idx

def run_markov_simulation(transition_matrix, initial_state_probs, n_cycles, discount_rate=0.035):
    """
    Run Markov simulation over multiple cycles.
    
    Args:
        transition_matrix: State transition probability matrix
        initial_state_probs: Initial distribution over states
        n_cycles: Number of cycles to simulate
        discount_rate: Annual discount rate
        
    Returns:
        DataFrame with state occupancy over cycles
    """
    n_states = transition_matrix.shape[0]
    state_history = np.zeros((n_cycles + 1, n_states))
    
    # Set initial state distribution
    state_history[0] = initial_state_probs
    
    # Simulate transitions
    current_probs = initial_state_probs.copy()
    for cycle in range(1, n_cycles + 1):
        # Apply transition matrix
        current_probs = np.dot(current_probs, transition_matrix)
        state_history[cycle] = current_probs
    
    # Calculate discounted state occupancy (for QALY calculations)
    cycle_discount_factors = [(1 / (1 + discount_rate)) ** (cycle/12) for cycle in range(n_cycles + 1)]
    
    discounted_state_history = state_history.copy()
    for cycle in range(n_cycles + 1):
        discounted_state_history[cycle] *= cycle_discount_factors[cycle]
    
    return state_history, discounted_state_history, cycle_discount_factors

def calculate_markov_outcomes(state_history, utilities, costs, n_cycles):
    """
    Calculate QALYs and costs from Markov simulation.
    
    Args:
        state_history: State occupancy over cycles
        utilities: Utility values for each state
        costs: Cost values for each state
        n_cycles: Number of cycles
        
    Returns:
        Total QALYs and costs
    """
    total_qalys = 0
    total_costs = 0
    
    # Calculate per cycle
    for cycle in range(1, n_cycles + 1):  # Start from 1 (after initial state)
        for state_idx in range(len(utilities)):
            # Calculate QALYs and costs for this state in this cycle
            state_occupancy = state_history[cycle, state_idx]
            cycle_qalys = state_occupancy * utilities[state_idx] * (1/12)  # Monthly utility * fraction of year
            cycle_costs = state_occupancy * costs[state_idx] * (1/12)      # Monthly cost * fraction of year
            
            total_qalys += cycle_qalys
            total_costs += cycle_costs
    
    return total_qalys, total_costs

In [ ]:
# Define health states for our model
# States for TRD treatment model
health_states = ['Depressed', 'Remission', 'Relapse', 'Treatment Discontinued', 'Death']

# Define transition probabilities for each strategy
# These would be calibrated to real data in practice
strategies = ['ECT', 'IV-KA', 'PO-KA']

transition_params = {
    'ECT': {
        'depression_to_remission': 0.65,     # 65% chance of remission
        'remission_to_relapse': 0.05,       # 5% monthly relapse rate
        'remission_to_death': 0.001,        # Low death rate in remission
        'depression_to_death': 0.005,       # Higher death rate in depression
        'relapse_to_remission': 0.40,       # 40% chance of return to remission
        'treat_discontinue_to_depression': 0.85,  # High chance of returning to depression
        'other_transitions_baseline': 0.02,
    },
    
    'IV-KA': {
        'depression_to_remission': 0.75,     # 75% chance of remission (higher efficacy)
        'remission_to_relapse': 0.04,       # 4% monthly relapse rate (slightly better)
        'remission_to_death': 0.001,
        'depression_to_death': 0.005,
        'relapse_to_remission': 0.50,       # 50% chance of return to remission
        'treat_discontinue_to_depression': 0.80,
        'other_transitions_baseline': 0.02,
    },
    
    'PO-KA': {
        'depression_to_remission': 0.70,     # 70% chance of remission
        'remission_to_relapse': 0.045,      # 4.5% monthly relapse rate
        'remission_to_death': 0.001,
        'depression_to_death': 0.005,
        'relapse_to_remission': 0.45,       # 45% chance of return to remission
        'treat_discontinue_to_depression': 0.82,
        'other_transitions_baseline': 0.02,
    }
}

# Define utilities and costs for each state
utilities = [0.55, 0.85, 0.60, 0.60, 0.0]  # Utility values for [Depressed, Remission, Relapse, TreatDisc, Death]
costs = [2000, 3000, 2200, 1800, 0]        # Monthly costs for [Depressed, Remission, Relapse, TreatDisc, Death]
          # Higher cost during active treatment

# Define initial state distribution
initial_state_probs = np.array([1.0, 0.0, 0.0, 0.0, 0.0])  # All patients start in 'Depressed' state

# Run Markov simulation for each strategy
markov_results = []
n_cycles = 120  # 120 months = 10 years
discount_rate = 0.035  # 3.5% annual discount rate

for strategy in strategies:
    params = transition_params[strategy]
    
    # Define transition probabilities based on strategy parameters
    trans_probs = {
        ('Depressed', 'Remission'): params['depression_to_remission'],
        ('Depressed', 'Depressed'): 1 - params['depression_to_remission'] - params['depression_to_death'] - params['other_transitions_baseline'],
        ('Depressed', 'Death'): params['depression_to_death'],
        ('Depressed', 'Treatment Discontinued'): params['other_transitions_baseline'],
        
        ('Remission', 'Relapse'): params['remission_to_relapse'],
        ('Remission', 'Remission'): 1 - params['remission_to_relapse'] - params['remission_to_death'] - params['other_transitions_baseline'],
        ('Remission', 'Death'): params['remission_to_death'],
        ('Remission', 'Treatment Discontinued'): params['other_transitions_baseline'],
        
        ('Relapse', 'Remission'): params['relapse_to_remission'],
        ('Relapse', 'Relapse'): 1 - params['relapse_to_remission'] - params['depression_to_death'] - params['other_transitions_baseline'],
        ('Relapse', 'Death'): params['depression_to_death'],  # Same mortality risk as depression
        ('Relapse', 'Treatment Discontinued'): params['other_transitions_baseline'],
        
        ('Treatment Discontinued', 'Depressed'): params['treat_discontinue_to_depression'],
        ('Treatment Discontinued', 'Treatment Discontinued'): 1 - params['treat_discontinue_to_depression'] - params['other_transitions_baseline'],
        ('Treatment Discontinued', 'Death'): params['other_transitions_baseline'],
        
        ('Death', 'Death'): 1.0,  # Absorbing state
    }
    
    # Create transition matrix
    tm, state_idx = create_markov_transition_matrix(health_states, trans_probs)
    
    # Run simulation
    state_history, discounted_state_history, discount_factors = \
        run_markov_simulation(tm, initial_state_probs, n_cycles, discount_rate)
    
    # Calculate outcomes
    total_qalys, total_costs = calculate_markov_outcomes(state_history, utilities, costs, n_cycles)
    
    # Calculate discounted outcomes
    disc_total_qalys, disc_total_costs = calculate_markov_outcomes(discounted_state_history, utilities, costs, n_cycles)
    
    markov_results.append({
        'strategy': strategy,
        'transition_matrix': tm,
        'state_history': state_history,
        'discounted_state_history': discounted_state_history,
        'n_cycles': n_cycles,
        'total_qalys': total_qalys,
        'total_costs': total_costs,
        'disc_total_qalys': disc_total_qalys,
        'disc_total_costs': disc_total_costs,
        'icer': None,  # Will calculate after all strategies
        'nmbr': None   # Net monetary benefit ratio
    })

# Calculate ICERs with ECT as the reference
ref_strategy_idx = next(i for i, r in enumerate(markov_results) if r['strategy'] == 'ECT')
ref_qalys = markov_results[ref_strategy_idx]['disc_total_qalys']
ref_costs = markov_results[ref_strategy_idx]['disc_total_costs']

wtp_threshold = 50000  # $50,000 per QALY

for i, result in enumerate(markov_results):
    if i == ref_strategy_idx:
        continue  # Skip reference strategy
    
    delta_qalys = result['disc_total_qalys'] - ref_qalys
    delta_costs = result['disc_total_costs'] - ref_costs
    
    if delta_qalys != 0:
        icer = delta_costs / delta_qalys
    else:
        icer = float('inf') if delta_costs > 0 else float('-inf')
    
    # Calculate net monetary benefit
    nmbr = (result['disc_total_qalys'] * wtp_threshold - result['disc_total_costs']) - \
           (ref_qalys * wtp_threshold - ref_costs)
    
    markov_results[i]['icer'] = icer
    markov_results[i]['nmbr'] = nmbr

# Print results
print("MARKOV MODEL RESULTS")
print("="*60)
for result in markov_results:
    print(f"\n{result['strategy']}:\n" + "-"*30)
    print(f"  Total QALYs (undiscounted): {result['total_qalys']:.4f}")
    print(f"  Total Costs (undiscounted): ${result['total_costs']:,.2f}")
    print(f"  Discounted QALYs: {result['disc_total_qalys']:.4f}")
    print(f"  Discounted Costs: ${result['disc_total_costs']:,.2f}")
    
    if result['strategy'] != 'ECT':  # Not the reference
        print(f"  ICER vs ECT: ${result['icer']:,.2f}/QALY")
        print(f"  NMBR vs ECT: ${result['nmbr']:,.2f}")
        
        # Determine cost-effectiveness
        if result['icer'] <= wtp_threshold and result['nmbr'] > 0:
            print(f"  Status: Cost-effective vs ECT at WTP of ${wtp_threshold:,}/QALY")
        elif result['nmbr'] > 0:
            print(f"  Status: Dominates ECT (more effective, same or lower cost)")
        else:
            print(f"  Status: Not cost-effective vs ECT at WTP of ${wtp_threshold:,}/QALY")

In [ ]:
# Visualize Markov model results
fig, ax = plt.subplots(2, 2, figsize=(16, 12))

# 1. State occupancy over time for each strategy
colors = ['blue', 'green', 'orange']
for i, result in enumerate(markov_results):
    strategy_data = result['state_history']
    for state_idx, state in enumerate(health_states):
        ax[0, 0].plot(range(result['n_cycles']+1), strategy_data[:, state_idx], 
                      label=f"{result['strategy']} - {state}", color=colors[i], 
                      linestyle=['-', '--', '-.', ':'][state_idx], alpha=0.7)

ax[0, 0].set_xlabel('Months')
ax[0, 0].set_ylabel('Proportion in State')
ax[0, 0].set_title('State Occupancy Over Time')
ax[0, 0].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax[0, 0].grid(True, alpha=0.3)

# 2. Discounted state occupancy
for i, result in enumerate(markov_results):
    strategy_data = result['discounted_state_history']
    for state_idx, state in enumerate(health_states):
        ax[0, 1].plot(range(result['n_cycles']+1), strategy_data[:, state_idx], 
                      label=f"{result['strategy']} - {state}", color=colors[i], 
                      linestyle=['-', '--', '-.', ':'][state_idx], alpha=0.7)

ax[0, 1].set_xlabel('Months')
ax[0, 1].set_ylabel('Discounted Proportion in State')
ax[0, 1].set_title('Discounted State Occupancy Over Time')
ax[0, 1].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax[0, 1].grid(True, alpha=0.3)

# 3. QALYs and Costs comparison
strategies_list = [r['strategy'] for r in markov_results]
qalys_list = [r['disc_total_qalys'] for r in markov_results]
costs_list = [r['disc_total_costs'] for r in markov_results]

x = np.arange(len(strategies_list))
width = 0.35

ax1_twin = ax[1, 0].twinx()
bars1 = ax[1, 0].bar(x - width/2, qalys_list, width, label='Discounted QALYs', 
                     color=[c+'1' if c != 'orange' else 'darkorange' for c in colors], alpha=0.7)
bars2 = ax1_twin.bar(x + width/2, costs_list, width, label='Discounted Costs', 
                     color=[c+'2' if c != 'orange' else 'orangered' for c in colors], alpha=0.7)

ax[1, 0].set_xlabel('Strategy')
ax[1, 0].set_ylabel('Discounted QALYs', color='blue')
ax1_twin.set_ylabel('Discounted Costs ($)', color='red')
ax[1, 0].set_title('Discounted QALYs and Costs by Strategy')
ax[1, 0].set_xticks(x)
ax[1, 0].set_xticklabels(strategies_list)
ax[1, 0].grid(True, alpha=0.3)

# 4. ICER vs ECT
non_ref_strategies = [r for r in markov_results if r['strategy'] != 'ECT']
if non_ref_strategies:
    icer_strategies = [r['strategy'] for r in non_ref_strategies]
    icer_values = [r['icer'] for r in non_ref_strategies]
    nmbr_values = [r['nmbr'] for r in non_ref_strategies]
    
    x_icer = np.arange(len(icer_strategies))
    
    ax4_twin = ax[1, 1].twinx()
    bars3 = ax[1, 1].bar(x_icer, icer_values, width=0.4, label='ICER', 
                         color=[c for i, c in enumerate(colors) if strategies[i] != 'ECT'], alpha=0.7)
    bars4 = ax4_twin.bar(x_icer + 0.4, nmbr_values, width=0.4, label='NMBR', 
                        color=[c for i, c in enumerate(['lightblue', 'lightgreen', 'moccasin']) if strategies[i] != 'ECT'], 
                        alpha=0.7)
    
    ax[1, 1].axhline(y=wtp_threshold, color='red', linestyle='--', label=f'WTP Threshold (${wtp_threshold:,})')
    ax[1, 1].set_xlabel('Strategy vs ECT')
    ax[1, 1].set_ylabel('ICER ($/QALY)', color='blue')
    ax4_twin.set_ylabel('Net Monetary Benefit Difference ($)', color='purple')
    ax[1, 1].set_title('ICER and NMBR vs ECT')
    ax[1, 1].set_xticks(x_icer + 0.2)
    ax[1, 1].set_xticklabels(icer_strategies)
    ax[1, 1].legend(loc='upper left')
    ax4_twin.legend(loc='upper right')
    ax[1, 1].grid(True, alpha=0.3)
else:
    ax[1, 1].text(0.5, 0.5, 'No comparison strategies', 
                  horizontalalignment='center', verticalalignment='center',
                  transform=ax[1, 1].transAxes)
    ax[1, 1].set_title('ICER and NMBR vs ECT')

plt.tight_layout()
plt.show()

In [ ]:
# Visualize transition matrices
fig, ax = plt.subplots(1, 3, figsize=(18, 5))

for i, result in enumerate(markov_results):
    tm = result['transition_matrix']
    
    im = ax[i].imshow(tm, cmap='viridis', aspect='auto')
    ax[i].set_xticks(np.arange(len(health_states)))
    ax[i].set_yticks(np.arange(len(health_states)))
    ax[i].set_xticklabels(health_states, rotation=45)
    ax[i].set_yticklabels(health_states)
    ax[i].set_title(f'Transition Matrix: {result["strategy"]}')
    
    # Add text annotations
    for j in range(len(health_states)):
        for k in range(len(health_states)):
            text = ax[i].text(k, j, f'{tm[j, k]:.3f}',
                             ha="center", va="center", color="white" if tm[j, k] < 0.5 else "black")
    
    plt.colorbar(im, ax=ax[i])

plt.tight_layout()
plt.show()

# Create a summary table
summary_df = pd.DataFrame([
    {
        'Strategy': r['strategy'],
        'Discounted QALYs': r['disc_total_qalys'],
        'Discounted Costs': r['disc_total_costs'],
        'ICER vs ECT': r['icer'] if r['strategy'] != 'ECT' else 'Reference',
        'NMBR vs ECT': r['nmbr'] if r['strategy'] != 'ECT' else 'Reference',
        'Cost-Effective': 'Yes' if r['strategy'] == 'ECT' or (r['icer'] <= wtp_threshold and r['nmbr'] > 0) else 'No'
    } for r in markov_results
])

print("\nMARKOV MODEL SUMMARY TABLE")
print("="*80)
print(summary_df.round(4))

In [ ]:
# Perform sensitivity analysis on discount rate
discount_rates = [0.0, 0.03, 0.035, 0.05, 0.07]  # Different discount rates
sensitivity_results = []

for dr in discount_rates:
    for strategy in strategies:
        params = transition_params[strategy]
        
        # Define transition probabilities
        trans_probs = {
            ('Depressed', 'Remission'): params['depression_to_remission'],
            ('Depressed', 'Depressed'): 1 - params['depression_to_remission'] - params['depression_to_death'] - params['other_transitions_baseline'],
            ('Depressed', 'Death'): params['depression_to_death'],
            ('Depressed', 'Treatment Discontinued'): params['other_transitions_baseline'],
            
            ('Remission', 'Relapse'): params['remission_to_relapse'],
            ('Remission', 'Remission'): 1 - params['remission_to_relapse'] - params['remission_to_death'] - params['other_transitions_baseline'],
            ('Remission', 'Death'): params['remission_to_death'],
            ('Remission', 'Treatment Discontinued'): params['other_transitions_baseline'],
            
            ('Relapse', 'Remission'): params['relapse_to_remission'],
            ('Relapse', 'Relapse'): 1 - params['relapse_to_remission'] - params['depression_to_death'] - params['other_transitions_baseline'],
            ('Relapse', 'Death'): params['depression_to_death'],  # Same mortality risk as depression
            ('Relapse', 'Treatment Discontinued'): params['other_transitions_baseline'],
            
            ('Treatment Discontinued', 'Depressed'): params['treat_discontinue_to_depression'],
            ('Treatment Discontinued', 'Treatment Discontinued'): 1 - params['treat_discontinue_to_depression'] - params['other_transitions_baseline'],
            ('Treatment Discontinued', 'Death'): params['other_transitions_baseline'],
            
            ('Death', 'Death'): 1.0,  # Absorbing state
        }
        
        # Create transition matrix
        tm, state_idx = create_markov_transition_matrix(health_states, trans_probs)
        
        # Run simulation
        state_history, discounted_state_history, discount_factors = \
            run_markov_simulation(tm, initial_state_probs, n_cycles, dr)
        
        # Calculate outcomes
        total_qalys, total_costs = calculate_markov_outcomes(state_history, utilities, costs, n_cycles)
        disc_total_qalys, disc_total_costs = calculate_markov_outcomes(discounted_state_history, utilities, costs, n_cycles)
        
        sensitivity_results.append({
            'strategy': strategy,
            'discount_rate': dr,
            'disc_qalys': disc_total_qalys,
            'disc_costs': disc_total_costs
        })

sens_df = pd.DataFrame(sensitivity_results)

# Plot sensitivity analysis
fig, ax = plt.subplots(1, 2, figsize=(16, 6))

# QALYs vs discount rate
for strategy in strategies:
    strat_data = sens_df[sens_df['strategy'] == strategy]
    ax[0].plot(strat_data['discount_rate'], strat_data['disc_qalys'], 
               label=strategy, marker='o', linewidth=2, markersize=6)

ax[0].set_xlabel('Discount Rate')
ax[0].set_ylabel('Discounted QALYs')
ax[0].set_title('Sensitivity Analysis: Discounted QALYs vs Discount Rate')
ax[0].legend()
ax[0].grid(True, alpha=0.3)

# Costs vs discount rate
for strategy in strategies:
    strat_data = sens_df[sens_df['strategy'] == strategy]
    ax[1].plot(strat_data['discount_rate'], strat_data['disc_costs'], 
               label=strategy, marker='s', linewidth=2, markersize=6)

ax[1].set_xlabel('Discount Rate')
ax[1].set_ylabel('Discounted Costs ($AUD)')
ax[1].set_title('Sensitivity Analysis: Discounted Costs vs Discount Rate')
ax[1].legend()
ax[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Perform deterministic sensitivity analysis on transition probabilities
def run_markov_dsa(base_params, param_name, param_range, strategy, n_cycles=120):
    """Run deterministic sensitivity analysis on a specific parameter."""
    results = []
    
    for value in param_range:
        # Create modified parameter set
        mod_params = base_params.copy()
        mod_params[param_name] = value
        
        # Define transition probabilities based on modified parameters
        trans_probs = {
            ('Depressed', 'Remission'): mod_params['depression_to_remission'],
            ('Depressed', 'Depressed'): 1 - mod_params['depression_to_remission'] - mod_params['depression_to_death'] - mod_params['other_transitions_baseline'],
            ('Depressed', 'Death'): mod_params['depression_to_death'],
            ('Depressed', 'Treatment Discontinued'): mod_params['other_transitions_baseline'],
            
            ('Remission', 'Relapse'): mod_params['remission_to_relapse'],
            ('Remission', 'Remission'): 1 - mod_params['remission_to_relapse'] - mod_params['remission_to_death'] - mod_params['other_transitions_baseline'],
            ('Remission', 'Death'): mod_params['remission_to_death'],
            ('Remission', 'Treatment Discontinued'): mod_params['other_transitions_baseline'],
            
            ('Relapse', 'Remission'): mod_params['relapse_to_remission'],
            ('Relapse', 'Relapse'): 1 - mod_params['relapse_to_remission'] - mod_params['depression_to_death'] - mod_params['other_transitions_baseline'],
            ('Relapse', 'Death'): mod_params['depression_to_death'],
            ('Relapse', 'Treatment Discontinued'): mod_params['other_transitions_baseline'],
            
            ('Treatment Discontinued', 'Depressed'): mod_params['treat_discontinue_to_depression'],
            ('Treatment Discontinued', 'Treatment Discontinued'): 1 - mod_params['treat_discontinue_to_depression'] - mod_params['other_transitions_baseline'],
            ('Treatment Discontinued', 'Death'): mod_params['other_transitions_baseline'],
            
            ('Death', 'Death'): 1.0,  # Absorbing state
        }
        
        # Create transition matrix
        tm, state_idx = create_markov_transition_matrix(health_states, trans_probs)
        
        # Run simulation
        state_history, discounted_state_history, discount_factors = \
            run_markov_simulation(tm, initial_state_probs, n_cycles, 0.035)
        
        # Calculate outcomes
        disc_total_qalys, disc_total_costs = \
            calculate_markov_outcomes(discounted_state_history, utilities, costs, n_cycles)
        
        results.append({
            'param_value': value,
            'disc_qalys': disc_total_qalys,
            'disc_costs': disc_total_costs
        })
    
    return pd.DataFrame(results)

# Run DSA on key parameters for IV-KA (most effective strategy)
param_ranges = {
    'depression_to_remission': np.linspace(0.5, 0.9, 10),
    'remission_to_relapse': np.linspace(0.02, 0.1, 10),
    'relapse_to_remission': np.linspace(0.3, 0.7, 10)
}

dsa_results = {}
for param_name, param_range in param_ranges.items():
    dsa_results[param_name] = run_markov_dsa(
        transition_params['IV-KA'], param_name, param_range, 'IV-KA'
    )

# Plot DSA results
fig, ax = plt.subplots(1, 3, figsize=(18, 6))

for i, (param_name, results) in enumerate(dsa_results.items()):
    ax[i].plot(results['param_value'], results['disc_qalys'], label='QALYs', marker='o', linewidth=2)
    ax_twin = ax[i].twinx()
    ax_twin.plot(results['param_value'], results['disc_costs'], label='Costs', 
                 color='red', marker='s', linewidth=2)

    ax[i].set_xlabel(param_name.replace('_', ' ').title())
    ax[i].set_ylabel('Discounted QALYs', color='blue')
    ax_twin.set_ylabel('Discounted Costs ($)', color='red')
    ax[i].set_title(f'Deterministic SA: {param_name.replace("_", " ").title()}')
    ax[i].grid(True, alpha=0.3)
    
    # Add legends
    ax[i].legend(loc='upper left')
    ax_twin.legend(loc='lower right')

plt.tight_layout()
plt.show()

In [ ]:
# Generate Markov model recommendations
print("MARKOV MODEL RECOMMENDATIONS")
print("="*70)

# Identify preferred strategy based on NMB at WTP threshold
wtp_threshold = 50000
best_strategy = None
best_nmbr = float('-inf')

for result in markov_results:
    if result['strategy'] == 'ECT':
        # Calculate NMB for reference strategy
        ref_nmb = result['disc_total_qalys'] * wtp_threshold - result['disc_total_costs']
        result['nmb'] = ref_nmb
    else:
        # Calculate NMB difference from reference
        result['nmb'] = ref_nmb + result['nmbr']
    
    if result['nmb'] > best_nmbr:
        best_nmbr = result['nmb']
        best_strategy = result['strategy']

print(f"Recommended Strategy: {best_strategy} (NMB: ${best_nmbr:,.2f})")

# Identify cost-effective strategies
print(f"\nCost-Effective Strategies at WTP of ${wtp_threshold:,}/QALY:")
ce_strategies = []
for result in markov_results:
    if result['strategy'] == 'ECT':
        ce_strategies.append((result['strategy'], result['nmb']))
    else:
        # Check if strategy is cost-effective vs ECT
        if result['nmbr'] >= 0:  # NMB difference is non-negative
            ce_strategies.append((result['strategy'], result['nmb']))

ce_strategies.sort(key=lambda x: x[1], reverse=True)  # Sort by NMB
for i, (strategy, nmb) in enumerate(ce_strategies):
    print(f"  {i+1}. {strategy} (NMB: ${nmb:,.2f})")

# Calculate incremental cost-effectiveness
print(f"\nINCREMENTAL ANALYSIS (")
ref_qalys = next(r['disc_total_qalys'] for r in markov_results if r['strategy'] == 'ECT')
ref_costs = next(r['disc_total_costs'] for r in markov_results if r['strategy'] == 'ECT')

print(f"Reference: ECT (QALYs: {ref_qalys:.4f}, Costs: ${ref_costs:,.2f})")
print(f"WTP Threshold: ${wtp_threshold:,}/QALY")
print("-"*50)

for result in markov_results:
    if result['strategy'] == 'ECT':
        continue
        
    inc_qalys = result['disc_total_qalys'] - ref_qalys
    inc_costs = result['disc_total_costs'] - ref_costs
    icer = result['icer']
    nmbr = result['nmbr']
    
    ce_status = "Cost-Effective" if (icer <= wtp_threshold and nmbr > 0) else "Not Cost-Effective"
    
    print(f"{result['strategy']}: ")
    print(f"  Incremental QALYs: {inc_qalys:+.4f}")
    print(f"  Incremental Costs: ${inc_costs:+,.2f}")
    print(f"  ICER: ${icer:,.2f}/QALY")
    print(f"  NMB Difference: ${nmbr:,.2f}")
    print(f"  Status: {ce_status}")
    print()

## Next Steps

1. Calibrate transition probabilities to real-world data
2. Incorporate time-dependent transitions (semi-Markov)
3. Add more health states for greater granularity
4. Perform probabilistic sensitivity analysis
5. Validate model against external datasets